## **Data Preprocessing**

In [1]:
# Install required libraries
# !pip install fastapi nest-asyncio uvicorn joblib xgboost scikit-learn pandas requests pyngrok

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def load_data(app_file, cred_file):
    """
    Loads application & credit data, merges them, and handles missing values.
    """
    try:
        # Load datasets
        appsamp_df = pd.read_csv(app_file)
        credfeat_df = pd.read_csv(cred_file)

        # Merge datasets on UID
        df = pd.merge(appsamp_df, credfeat_df, on="UID", how="left")

        # Encode categorical column (EmploymentType)
        df["EmploymentType"] = LabelEncoder().fit_transform(df["EmploymentType"])

        # Replace missing values with median
        df.fillna(df.median(), inplace=True)

        # Define features & target
        X = df.drop(columns=["UID", "Success"])  # Features
        y = df["Success"]  # Target variable

        print("Data Preprocessing Completed!")
        return X, y
    except Exception as e:
        print(f"Data Loading Error: {e}")
        return None, None

# Call function
X, y = load_data("application_samples.csv", "credit_features.csv")

Data Preprocessing Completed!


### **Model Training & Evaluation**

In [4]:
# Train XGBoost Model
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [5]:
def train_model(X, y, model_filename="loan_success_model.joblib"):
    """
    Trains an XGBoost classifier and saves the model.
    """
    try:
        # Split data into training & testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

        # Train XGBoost model
        model = xgb.XGBClassifier(objective="binary:logistic", eval_metric="auc")
        model.fit(X_train, y_train)

        # Save trained model
        joblib.dump(model, model_filename)
        print(f"Model trained and saved as {model_filename}")

        return model, X_test, y_test
    except Exception as e:
        print(f"Model Training Error: {e}")
        return None, None, None

# Train and save model
model, X_test, y_test = train_model(X, y)

Model trained and saved as loan_success_model.joblib


In [6]:
# Evaluating Model Performance
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the model and prints performance metrics.
    """
    try:
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]

        metrics = {
            "Accuracy": accuracy_score(y_test, y_pred),
            "Precision": precision_score(y_test, y_pred),
            "Recall": recall_score(y_test, y_pred),
            "F1-Score": f1_score(y_test, y_pred),
            "AUC-ROC": roc_auc_score(y_test, y_prob),
        }

        print("Model Evaluation Metrics:", metrics)
        return metrics
    except Exception as e:
        print(f"Model Evaluation Error: {e}")
        return None

# Evaluate model
evaluate_model(model, X_test, y_test)


Model Evaluation Metrics: {'Accuracy': 0.8967984934086629, 'Precision': 0.5089820359281437, 'Recall': 0.30685920577617326, 'F1-Score': 0.38288288288288286, 'AUC-ROC': 0.8515528930964649}


{'Accuracy': 0.8967984934086629,
 'Precision': 0.5089820359281437,
 'Recall': 0.30685920577617326,
 'F1-Score': 0.38288288288288286,
 'AUC-ROC': 0.8515528930964649}

### **Model Inferencing**

In [7]:
# Deploying FastAPI
import uvicorn
import requests
import threading
import time
import pandas as pd
import joblib
import os
from pyngrok import ngrok
from pyngrok.exception import PyngrokNgrokError
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from fastapi import FastAPI

In [8]:
nest_asyncio.apply()

app = FastAPI()

# Load trained model
model = joblib.load("loan_success_model.joblib")

In [9]:
# Authenticate Ngrok
from google.colab import userdata

NGROK_AUTHTOKEN = userdata.get("NGROK_AUTHTOKEN")

if NGROK_AUTHTOKEN:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("Ngrok authentication successful!")
else:
    print("Ngrok authtoken not found! Set it in Colab secrets.")

#Load Trained Model
try:
    model = joblib.load("loan_success_model.joblib")
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

Ngrok authentication successful!
Model loaded successfully!


In [10]:
# Define Pydantic Model for Request Validation
class LoanApplicationData(BaseModel):
    Amount: float
    Term: float
    EmploymentType: float
    ALL_AgeOfOldestAccount: float
    ALL_AgeOfYoungestAccount: float
    ALL_Count: float
    ALL_CountActive: float
    ALL_CountClosedLast12Months: float
    ALL_CountDefaultAccounts: float
    ALL_CountOpenedLast12Months: float
    ALL_CountSettled: float
    ALL_MeanAccountAge: float
    ALL_SumCurrentOutstandingBal: float
    ALL_SumCurrentOutstandingBalExcMtg: float
    ALL_TimeSinceMostRecentDefault: float
    ALL_WorstPaymentStatusActiveAccounts: float

# Define API Endpoint With Prediction
@app.post("/predict/")
def predict(data: LoanApplicationData):
    """Predicts loan application success using the trained model."""
    if model is None:
        return {"error": "Model not loaded"}

    try:
        df = pd.DataFrame([data.model_dump()])  # Convert Pydantic model to DataFrame
        prediction = model.predict(df)
        return {"prediction": int(prediction[0])}  # Return prediction as integer
    except Exception as e:
        return {"error": str(e)}

## **FastAPI Inference**

In [11]:
# Start FastAPI Server in a Background Thread
def run_api():
    """Starts FastAPI with Uvicorn in a separate thread"""
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_api, daemon=True)
thread.start()

# Give server time to start
time.sleep(3)
print("FastAPI server started successfully!")

# Kill Existing Ngrok Process Before Creating a New One
def kill_ngrok():
    """Terminates any existing Ngrok processes to prevent conflicts"""
    try:
        os.system("pkill -f ngrok")  # Kill all running Ngrok instances
        time.sleep(2)  # Allow some time for cleanup
        print("Existing Ngrok processes killed.")
    except Exception as e:
        print(f"Error killing Ngrok processes: {e}")

INFO:     Started server process [53632]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FastAPI server started successfully!


In [12]:
# Start Ngrok and Retrieve Public URL
def get_ngrok_url():
    """Starts Ngrok if not running and retrieves public URL"""
    try:
        kill_ngrok()  # Ensure no duplicate tunnels
        public_url = ngrok.connect(8000)  # Start a new tunnel
        print(f"Ngrok Tunnel Created: {public_url}")
        return public_url
    except PyngrokNgrokError as e:
        print(f"Error creating Ngrok tunnel: {e}")
        print("Ensure Ngrok authentication is correct and no tunnel limit exceeded.")
        return None

# Retrieve or create Ngrok URL
ngrok_url = get_ngrok_url()

Existing Ngrok processes killed.
Ngrok Tunnel Created: NgrokTunnel: "https://7b32-34-106-92-233.ngrok-free.app" -> "http://localhost:8000"


In [13]:
# Send a Test Request to Verify API with Prediction
data = {
    "Amount": 7000, "Term": 48, "EmploymentType": 1,
    "ALL_AgeOfOldestAccount": 199, "ALL_AgeOfYoungestAccount": 4,
    "ALL_Count": 11, "ALL_CountActive": 8, "ALL_CountClosedLast12Months": 0,
    "ALL_CountDefaultAccounts": 0, "ALL_CountOpenedLast12Months": 3,
    "ALL_CountSettled": 3, "ALL_MeanAccountAge": 69.73,
    "ALL_SumCurrentOutstandingBal": 124050, "ALL_SumCurrentOutstandingBalExcMtg": 1186,
    "ALL_TimeSinceMostRecentDefault": -1, "ALL_WorstPaymentStatusActiveAccounts": 0
}

if ngrok_url:
    # Extract public URL as a string
    api_url = f"{ngrok_url.public_url}/predict/"  # Use public_url attribute
    response = requests.post(api_url, json=data)
    print("API Response:", response.json())  # Now returns model prediction
else:
    print("Ngrok URL not available, cannot send request!")

INFO:     34.106.92.233:0 - "POST /predict/ HTTP/1.1" 200 OK
API Response: {'prediction': 1}
